## import library
- http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
- http://learningtensorflow.com/index.html
- http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

# 0. Tensorflow basic

- **tf.constant(), tf.Variable()**: 그래프의 객체
- **tf.Session()/ tf.InteractiveSession()**: 그래프를 시작
- **sess.run(c)**: 텐서 'c'를 계산 -> sess.close() 해야함
- **with 절**: tf.Session()은 with 절과 사용, with 절에서 sess.close() 생략가능
- **c.eval()**: sess.run(c)와 같음

### - **tf.variable_scope()**
- 모델에서 필요한 변수들을 관리하는 클래스(변수 범위 만들기)
- tf.get_variable(): 직접호출 없이 변수를 가져오거나 생성(initializer 사용)
- reuse = False: '현재 variable scope 이름 + 제공된 name'이 없으면 생성
- reuse = True: '현재 variable scope 이름 + 제공된 name'이 있으면 반환
- reuse = AUTO_REUSE: 둘다

# 2. RNN Training

In [68]:
tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello

### Initialize (input_data, output_data, parameters)

In [136]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
# None자리: batch size -> 1이라서 생략
print(X)
print(Y)

Tensor("Placeholder:0", shape=(?, 6, 5), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 6), dtype=int32)


### Create Cell

In [137]:
# LSTM cell 사용
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

# RNN cell 사용
#x_data2 = np.array([[x_one_hot[0][0],x_one_hot[0][1],x_one_hot[0][2],x_one_hot[0][3],
#                   x_one_hot[0][4],x_one_hot[0][5]]], dtype=np.float32)
#print(x_data2.shape)
#cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
#outputs, _states = tf.nn.dynamic_rnn(cell, x_data2, dtype=tf.float32)

### Train RNN (initialization)

In [139]:
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
# 학습을 위한 그래프 setting 완료

### Training

In [140]:
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.6061102 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
1 loss: 1.5172483 prediction:  [[3 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  leelll
2 loss: 1.4458308 prediction:  [[3 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llelll
3 loss: 1.3757234 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
4 loss: 1.3051718 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
5 loss: 1.241162 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
6 loss: 1.1981349 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
7 loss: 1.1576614 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
8 loss: 1.1198928 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
9 loss: 1.0877069 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predi